In [2]:
import matplotlib.pyplot as plt
import numpy as np
import cv2

In [3]:
import yolo
YOLO_WEIGHTS = './config/yolov4_Custom_FullData.weights'
YOLO_CFG = './config/yolov4TinyCustom.cfg'
YOLO_NET = cv2.dnn.readNet(YOLO_WEIGHTS, YOLO_CFG)

In [ ]:
def get_output_layers(net):
	layer_names = net.getLayerNames()
	output_layers = [layer_names[i[0] - 1] for i in net.getUnconnectedOutLayers()]
	return output_layers

def getYoloPredictions(frame, YOLO_NET):
	class_ids = []
	confidences = []
	boxes = []
	blob = cv2.dnn.blobFromImage(frame, 0.00392, (416,416), (0,0,0), True, crop=False)
	YOLO_NET.setInput(blob)
	outs = YOLO_NET.forward(get_output_layers(YOLO_NET))
	width = frame.shape[1]
	height = frame.shape[0]

	for out in outs:
		for detection in out:
			scores = detection[5:]
			class_id = np.argmax(scores)
			confidence = scores[class_id]
			if confidence > 0.5:
				center_x = int(detection[0] * width)
				center_y = int(detection[1] * height)
				w = int(detection[2] * width)
				h = int(detection[3] * height)
				x = center_x - w / 2
				y = center_y - h / 2
				class_ids.append(class_id)
				confidences.append(float(confidence))
				boxes.append([x, y, w, h])

	indices =  cv2.dnn.NMSBoxes(boxes, confidences,  0.5,  0.4)
	return indices, boxes

In [4]:
frame = cv2.imread('images/man.png')
predictions, boxes = yolo.getYoloPredictions(frame, YOLO_NET)

print(predictions)

for current in predictions:
    print('a')
    current = current[0]
    box = boxes[current]
    [x, y, w, h] = box
    label = str(names[current])

    cv2.rectangle(
        frame,
        (round(x), round(y)),
        (round(x+w), round(y+h)),
        (255, 0, 0),
        0
    )

    cv2.putText(
        frame,
        label,
        (100, 200),
        cv2_FONT_HERSHEY_SIMPLEX,
        2,
        (255, 0, 0),
        2,
        cv2, LINE_AA,
    )


# cv2.imshow("pee", frame)
# cv2.waitKey(0)
# plt.imshow(frame)

()
